In [1]:
import os
import json
from copy import copy

import pandas as pd
import numpy as np

# Get total & ICU staffed bed counts for every acute hospital facility in USA

Following methodology from this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5514420/

> Study Design and Data Sources

> We performed a repeated-measures time series analysis of US ICU bed supply during the 16-year period between 1996 and 2011. We obtained data on hospital characteristics and intensive care occupancy from the Centers for Medicare and Medicaid Services Hospital Cost Report Information System (HCRIS), a publicly available hospital-level database with detailed information on structural, organizational and cost data for all US hospitals. We excluded skilled nursing facilities, long term acute care hospitals, hospitals located in US territories and stand-alone pediatric hospitals (1, 2). We augmented the HCRIS data with data from the US Census Bureau’s 2010 urban-rural classification file which we used to designate hospitals as urban or rural by ZIP code (5).

> Variables

> The primary dependent variable was each hospital’s number of ICU beds compared to the previous year. We defined total ICU beds using the summed counts of four HCRIS bed categories that were available throughout the study interval: intensive care beds, surgical intensive care beds, cardiac intensive care beds and burn intensive care beds 

## Useful References & Links:

CMS Healthcare Cost Report Information System (HCRIS):
https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Cost-Reports/Hospital-2010-form

Hospital Facilities: 
- http://downloads.cms.gov/files/hcris/hosp10-reports.zip

Public Use File (annual, from 2015): 
- https://data.cms.gov/api/views/absp-nd3x/files/e0ca9126-8fd6-42ca-82bf-c2fe40bd4c0e?download=true&filename=CostReport_Documentation_2015_Final_Oct2019.xlsx
- https://www.cms.gov/files/zip/hospital-cost-report-public-use-file-2015.zip

![](https://www.resdac.org/sites/resdac.umn.edu/files/kb-images/Figure%204_2.png)

Direct link to 2018 reporting from hospitals: http://downloads.cms.gov/Files/hcris/HOSP10FY2018.zip


## Specific guidance for what we're doing here:
from CMS data research guide: https://www.resdac.org/articles/medicare-cost-report-data-structure

![alt text](https://www.resdac.org/sites/resdac.umn.edu/files/kb-images/Figure%205.PNG)

> In summary, the number of beds will be located in the numeric file. 

> To identify the number of beds for every report in numeric file, filter the records where the second column (Worksheet Indicator) is “S300001,” the third column (Line Number) is “01400”, and the fourth column (Column Number) is “00200”. 

> To identify the number of beds for a specific report submitted by a specific facility, filter the records by the “Record Report Number,” which is reported in Column 1. The Report Record Number for a specific facility can be found in the Report data file.

## Official Worksheet Definitions

from Provider Reimbursement Manual (https://www.cms.gov/Regulations-and-Guidance/Guidance/Manuals/Paper-Based-Manuals-Items/CMS021935):

Column 2--Refer to 42 CFR 412.105(b) and 69 FR 49093-49098 (August 11, 2004) to determine the facility bed count. Indicate the number of beds available for use by patients at the end of the cost reporting period.

A bed means an adult bed, pediatric bed, portion of inpatient labor/delivery/postpartum (LDP) room (also referred to as birthing room) bed when used for services other than labor and delivery, or newborn ICU bed (excluding newborn bassinets) maintained in a patient care area for lodging patients in acute, long term, or domiciliary areas of the hospital. Beds in post-anesthesia, post- operative recovery rooms, outpatient areas, emergency rooms, ancillary departments (however, see exception for labor and delivery department), nurses' and other staff residences, and other such areas that are regularly maintained and utilized for only a portion of the stay of patients (primarily for special procedures or not for inpatient lodging) are not termed a bed for these purposes. (See CMS Pub. 15-1, chapter 22, §2205.)

For cost reporting periods beginning prior to October 1, 2012, beds in distinct ancillary labor and delivery rooms and the proportion of LDP room (birthing room) beds used for labor and delivery services are not a bed for these purposes. (See 68 FR 45420 (August 1, 2003).)

For cost reporting periods beginning on or after October 1, 2012, in accordance with 77 FR 53411- 53413 (August 31, 2012), beds in distinct labor and delivery rooms, when occupied by an inpatient receiving IPPS-level acute care hospital services or when unoccupied, are considered to be part of a hospital’s inpatient available bed count in accordance with 42 CFR 412.105(b) and are to be reported on line 32. Furthermore, the proportion of the inpatient LDP room (birthing room) beds used for ancillary labor and delivery services is considered part of the hospital’s available bed count.

Column 8--Enter the number of inpatient days for all classes of patients for each component. Include organ acquisition and HMO days in this column. This amount will not equal the sum of columns 5 through 7, when the provider renders services to other than titles V, XVIII, or XIX patients.

Line 1--For cost reporting periods beginning before October 1, 2012, exclude from column 2 the portion of LDP room (birthing room) beds used for ancillary labor and delivery services, but include on this line beds used for routine adult and pediatric services (postpartum). In accordance with the instructions in 68 FR 45420 (August 1, 2003), compute this proportion (off the cost report) by multiplying the total number of occupied and unoccupied available beds in the LDP room by the percentage of time these beds were used for ancillary labor and delivery services. An example of how to calculate the “percentage of time” would be for a hospital to determine the number of hours for the cost reporting period during which each LDP room maternity patient received labor and delivery services and divide the sum of those hours for all such patients by the sum of the total hours (for both, ancillary labor and delivery services and for routine postpartum services) that all maternity patients spent in the LDP room during that cost reporting period. Alternatively, a hospital could calculate an average percentage of time maternity patients received ancillary labor and delivery services in an LDP room during a typical month.
For cost reporting periods beginning on or after October 1, 2012, include all the available LDP room (birthing room) beds in the available bed count in column 2. (See 77 FR 53411-53413 (August 31, 2012).) The proportion of available LDP room beds related to the ancillary labor and delivery services must not be excluded from column 2 for those cost reporting periods.
In columns 5, 6, 7 and 8, enter the number of adult and pediatric hospital days excluding the SNF and NF swing-bed, observation bed, and hospice days. In columns 6 and 7, also exclude HMO days. Do not include in column 6 Medicare Secondary Payer/Lesser of Reasonable Cost (MSP/LCC) days. Include these days only in column 8. However, do not include employee discount days in column 8.

Line 7--Enter the sum of lines 1, 5, and 6.
Lines 8 through 13--Enter the appropriate statistic applicable to each discipline for all programs.
Line 14--Enter the sum of lines 7 through 13 for columns 2 through 8, and for columns 12 through 15, enter the amount from line 1. For columns 9 through 11, enter the total for each from your records. Labor and delivery days (as defined in the instructions for Worksheet S-3, Part I, line 32) must not be included on this line.




## Methods

This notebook gathers the HCRIS information based on the following inputs:

- The hospital data at http://downloads.cms.gov/files/hcris/hosp10-reports.zip 
  - HOSPITAL10_PROVIDER_ID_INFO.CSV (`HOSPITAL10`): Provides facility level IDs, names and addresses
- The 2018 reporting data from http://downloads.cms.gov/Files/hcris/HOSP10FY2018.zip
  - hosp10_2018_RPT.CSV (`HOSP10_RPT`): Provides facility report information.
  - hosp10_2018_NMRC.CSV (`HOSP10_NMRC`): Contains the numeric column values that are linked back to the report data from above.
- A data dictionary at https://www.cms.gov/files/zip/hospital2010-documentation.zip
  - HCRIS_DataDictionary.csv: provides report column codes with titles.

It then takes the following steps:

- Join the `HOSPITAL10` data and `HOSP10_RPT` to get all information about a facility per report.
- Filter the `HOSP10_NMRC` to only those line numbers and columns we care about
  - Filter `Line Number` to keep information about Staffed Beds, Staffed Bed Days, and Inpatient Days AND
  - Filter `Column Number` to keep information concerning the numeric counts related to: 
    - Hospital Adult and Peds
    - Intensive Care Unit 
    - Coronary Care Unit
    - Burn ICU
    - Surgical ICU 
    - Total
- Join the filtered `HOSP10_NMRC` data to the previously joined `HOSPITAL10` and `HOSP10_RPT` data.
- Aggregate counts for 'Intensive Care Unit', 'Coronary Care Unit', 'Burn ICU', 'Surgical ICU' into 'ICU Total Staffed Beds',  'ICU Total Bed Days Available', 'ICU Total Inpatient Days'.
- Calculate 'ICU Occupancy Rate' and 'Total Bed Occupancy Rate' using the ratio of Inpatient Days to Bed Day Available for ICU and over all totals
- The resulting dataset will have information per Record per Facility. Some facility have multiple records. Choose the record with the 'Fiscal Year End Date' being the most recent.
- Drop problematic facilities, e.g. PARKVIEW MEDICAL CENTER which reports 2,290,239,239 total beds. 
- Join with the `usa_hospital_beds_hcris2018_geocoded.geojson` dataset based on 'Provider Number', generate a new GeoJSON file that contains all aggregated counts and other relevant HCRIS information with this joined data.


### Download and Format the HOSPITAL10 data

In [2]:
# get facility level IDs, names and addresses
if not os.path.exists('download_data/HOSPITAL10_PROVIDER_ID_INFO.CSV'):
    if not os.path.isdir('download_data'):
        os.makedirs('download_data')

    !wget -O download_data/hosp10-reports.zip http://downloads.cms.gov/files/hcris/hosp10-reports.zip
        
    !cd download_data && unzip -o hosp10-reports.zip
    
hosp_df = pd.read_csv('download_data/HOSPITAL10_PROVIDER_ID_INFO.CSV')
hosp_df.head()

,PROVIDER_NUMBER,FYB,FYE,STATUS,CTRL_TYPE,HOSP10_Name,Street_Addr,PO_Box,City,State,Zip_Code,County
0,10001,01-OCT-17,30-SEP-18,As Submitted,9,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,6987,DOTHAN,AL,36301,HOUSTON
1,10005,01-OCT-17,30-SEP-18,As Submitted,9,MARSHALL MEDICAL CENTER - SOUTH,2505 U.S. HIGHWAY 431,NaN,BOAZ,AL,35957-,MARSHALL
2,10006,01-JUL-18,30-JUN-19,As Submitted,4,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,818,FLORENCE,AL,35630,LAUDERDALE
3,10007,01-OCT-17,30-SEP-18,As Submitted,9,MIZELL MEMORIAL HOSPITAL,702 MAIN STREET,429,OPP,AL,36462-,COVINGTON
4,10008,01-JAN-18,31-DEC-18,As Submitted,4,CRENSHAW COMMUNITY HOSPITAL,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW


In [3]:
# provider num should be 6 char so need to zfill
hosp_df['PROVIDER_NUMBER'] = hosp_df['PROVIDER_NUMBER'].apply(lambda x: str(x).zfill(6))

# Rename this column to match up with reports
hosp_df = hosp_df.rename(columns={'PROVIDER_NUMBER': 'Provider Number'})

In [4]:
# Show all providers in San Francisco county
hosp_df[hosp_df['County'] == 'SAN FRANCISCO']

,Provider Number,FYB,FYE,STATUS,CTRL_TYPE,HOSP10_Name,Street_Addr,PO_Box,City,State,Zip_Code,County
399,050008,01-JAN-18,31-DEC-18,As Submitted,2,CPMC-R.K. DAVIES MEDICAL CENTER,601 DUBOCE AVE,NaN,SAN FRANCISCO,CA,94117-3389,SAN FRANCISCO
420,050047,01-JAN-18,31-DEC-18,As Submitted,2,CALIFORNIA PACIFIC MEDICAL CENTER,2333 BUCHANAN ST,NaN,SAN FRANCISCO,CA,94115-1925,SAN FRANCISCO
422,050055,01-JAN-18,31-DEC-18,As Submitted,2,CPMC - MISSION BERNAL CAMPUS,3555 CESAR CHAVEZ STREET,NaN,SAN FRANCISCO,CA,94110-4403,SAN FRANCISCO
435,050076,01-JAN-18,31-DEC-18,As Submitted,2,KFH - SAN FRANCISCO,2425 GEARY BOULEVARD,NaN,SAN FRANCISCO,CA,94115-,SAN FRANCISCO
482,050152,01-JUL-18,30-JUN-19,As Submitted,2,SAINT FRANCIS MEMORIAL HOSPITAL,900 HYDE STREET,NaN,SAN FRANCISCO,CA,94109,SAN FRANCISCO
507,050228,01-JUL-18,30-JUN-19,As Submitted,8,ZUCKERBERG SAN FRANCISCO GENERAL,1001 POTRERO AVENUE,NaN,SAN FRANCISCO,CA,94110-,SAN FRANCISCO
576,050407,01-JAN-18,31-DEC-18,As Submitted,2,CHINESE HOSPITAL,845 JACKSON STREET,NaN,SAN FRANCISCO,CA,94133-,SAN FRANCISCO
590,050454,01-JUL-18,30-JUN-19,As Submitted,10,UCSF MEDICAL CENTER,505 PARNASSUS,NaN,SAN FRANCISCO,CA,94143-0824,SAN FRANCISCO
592,050457,01-JUL-18,30-JUN-19,As Submitted,1,ST. MARYS MEDICAL CENTER,450 STANYAN STREET,NaN,SAN FRANCISCO,CA,94117,SAN FRANCISCO
654,050668,01-JUL-18,30-JUN-19,As Submitted,8,LAGUNA HONDA HOSPITAL,375 LAGUNA HONDA BLVD,NaN,SAN FRANCISCO,CA,94116-,SAN FRANCISCO


### Download the data dictionary and generate data mappings

In [5]:
if not os.path.exists('download_data/HCRIS_DataDictionary.csv'):
    !wget -O download_data/hospital2010-documentation.zip https://www.cms.gov/files/zip/hospital2010-documentation.zip
    !cd download_data && unzip -o hospital2010-documentation.zip

hcris_dict = pd.read_csv('download_data/HCRIS_DataDictionary.csv')
hcris_dict.head()

,Column Code,TABLES,SUBSYSTEM,Null/Not Null,Title,Description,Valid Entries
0,ADR_VNDR_CD,RPT,ALL,NaN,Automated Desk Review Vendor Code,Vendor for Fiscal Intermediary.,2 or A03 - E & Y ...
1,ALPHNMRC_ITM_TXT,ALPHA,ALL,NOT NULL,Alphanumeric Item Text,Provider reported alpha data.,Per Specification Table
2,CLMN_NUM,"ALPHA,NMRC",HOSP10,NOT NULL,Column Number,Valid Column Number defined as follows: xxxyy...,"Example: Column 1 = 00100, Column 1.01 = 00101"
3,CLMN_NUM,"ALPHA,NMRC",ALL BUT HOSP10,NOT NULL,Column Number,Valid Column Number defined as follows: xxyy ...,"Example: Column 1 = 0100, Column 1.01 = 0101"
4,FI_CREAT_DT,RPT,ALL,NaN,Fiscal Intermediary Create Date,Date the FI created the HCRIS file.,MM/DD/YYYY


In [6]:
data_dict = {c:t for c,t in zip(hcris_dict['Column Code'],hcris_dict['Title'])}
data_dict

{'ADR_VNDR_CD': 'Automated Desk Review Vendor Code',
 'ALPHNMRC_ITM_TXT': 'Alphanumeric Item Text',
 'CLMN_NUM': 'Column Number',
 'FI_CREAT_DT': 'Fiscal Intermediary Create Date',
 'FI_NUM': 'Fiscal Intermediary Number',
 'FI_RCPT_DT': 'Fiscal Intermediary Receipt Date',
 'FY_BGN_DT': 'Fiscal Year Begin Date',
 'FY_END_DT': 'Fiscal Year End Date',
 'INITL_RPT_SW': 'Initial Report Switch',
 'ITM_VAL_NUM': 'Item Value Number',
 'LAST_RPT_SW': 'Last Report Switch',
 'LINE_NUM': 'Line Number',
 'NPR_DT': 'Notice of Program Reimbursement Date',
 'NPI': 'National Provider Identifier',
 'PROC_DT': 'Process Date',
 'PRVDR_CTRL_TYPE_CD': 'Provider Control Type Code',
 'PRVDR_NUM': 'Provider Number',
 'RPT_REC_NUM': 'Report Record Number',
 'RPT_STUS_CD': 'Report Status Code',
 'SPEC_IND': 'Special Indicator',
 'TRNSMTL_NUM': 'The current transmittal or version number in effect for each sub-system.',
 'UTIL_CD': 'Utilization Code',
 'LABEL': 'Rollup label',
 'ITEM': 'Rollup value',
 'WKSHT_CD':

In [7]:
# Report Table file columns
rpt_columns = [
               'RPT_REC_NUM',
               'PRVDR_CTRL_TYPE_CD',
               'PRVDR_NUM',
               'NPI',
               'RPT_STUS_CD',
               'FY_BGN_DT',
               'FY_END_DT',
               'PROC_DT',
               'INITL_RPT_SW',
               'LAST_RPT_SW',
               'TRNSMTL_NUM',
               'FI_NUM',
               'ADR_VNDR_CD',
               'FI_CREAT_DT',
               'UTIL_CD',
               'NPR_DT',
               'SPEC_IND',
               'FI_RCPT_DT'
]
[data_dict[col] for col in rpt_columns]

['Report Record Number',
 'Provider Control Type Code',
 'Provider Number',
 'National Provider Identifier',
 'Report Status Code',
 'Fiscal Year Begin Date',
 'Fiscal Year End Date',
 'Process Date',
 'Initial Report Switch',
 'Last Report Switch',
 'The current transmittal or version number in effect for each sub-system.',
 'Fiscal Intermediary Number',
 'Automated Desk Review Vendor Code',
 'Fiscal Intermediary Create Date',
 'Utilization Code',
 'Notice of Program Reimbursement Date',
 'Special Indicator',
 'Fiscal Intermediary Receipt Date']

In [8]:
# Numerical Table file columns
nmrc_columns = [
             'RPT_REC_NUM',
             'WKSHT_CD',
             'LINE_NUM',
             'CLMN_NUM',
             'ITM_VAL_NUM'
]
[data_dict[col] for col in nmrc_columns]

['Report Record Number',
 'Worksheet Identifier',
 'Line Number',
 'Column Number',
 'Item Value Number']

In [9]:
# Maps to 'Line Number' in the numeric report
beds_dict = {
    'Hospital Adult and Peds': '00100',
    'Intensive Care Unit': '00800',
    'Coronary Care Unit': '00900',
    'Burn ICU': '01000',
    'Surgical ICU': '01100',
    'Total': '01400'
}

# Maps to 'Column Number' in numeric report
value_count_dict = {
    'Staffed Beds': '00200',
    'Bed Days Available': '00300',
    'Inpatient Days': '00800'
}

icu_beds = ['Intensive Care Unit', 'Coronary Care Unit', 'Burn ICU', 'Surgical ICU']
icu_staffed_beds_columns = ['{} Staffed Beds'.format(x) for x in icu_beds]
icu_bed_days_columns = ['{} Bed Days Available'.format(x) for x in icu_beds]
icu_inpatient_days_columns = ['{} Inpatient Days'.format(x) for x in icu_beds]

all_count_columns = [
    'ICU Total Staffed Beds', 
    'ICU Total Bed Days Available',
    'ICU Total Inpatient Days',
    'ICU Occupancy Rate',
    'Total Bed Occupancy Rate'
]

for bed_desc in beds_dict:
    for value_desc in value_count_dict:
        column_name = '{} {}'.format(bed_desc, value_desc)
        all_count_columns.append(column_name)

In [10]:
beds_dict_flip = {v:k for k,v in beds_dict.items()}
bedtype_list = list(beds_dict_flip.keys())

value_count_dict_flip = {v:k for k,v in value_count_dict.items()}
value_count_list = list(value_count_dict_flip.keys())

### Download the HCRIS report file

Currently using the data for FY2018, as the FY2019 only contains reports from 97 providers.

In [11]:
# get the HCRIS file for FY2018
if not os.path.exists('download_data/hosp10_2018_RPT.CSV'):
    !wget -O download_data/HOSP10FY2018.zip http://downloads.cms.gov/Files/hcris/HOSP10FY2018.zip
    !cd download_data && unzip -o HOSP10FY2018.zip
    
if not os.path.exists('download_data/hosp10_2019_RPT.CSV'):
    !wget -O download_data/HOSP10FY2019.zip http://downloads.cms.gov/Files/hcris/HOSP10FY2019.zip
    !cd download_data && unzip -o HOSP10FY2019.zip

# 2019 data only has 97 providers as opposed to 5704 in 2018 - Not sure why?
USE_2019 = False
    
if USE_2019:    
    hosp10_rpt_df = pd.read_csv('download_data/hosp10_2019_RPT.CSV', names=[data_dict[col] for col in rpt_columns], dtype={'Provider Number':object})
    hosp10_nmrc_df = pd.read_csv('download_data/hosp10_2019_NMRC.CSV',  names=[data_dict[col] for col in nmrc_columns], dtype={'Line Number':object, 'Column Number':object})
else:
    hosp10_rpt_df = pd.read_csv('download_data/hosp10_2018_RPT.CSV', names=[data_dict[col] for col in rpt_columns], dtype={'Provider Number':object})
    hosp10_nmrc_df = pd.read_csv('download_data/hosp10_2018_NMRC.CSV',  names=[data_dict[col] for col in nmrc_columns], dtype={'Line Number':object, 'Column Number':object})

In [12]:
hosp10_rpt_df[hosp10_rpt_df['Provider Number'] == '010032']

,Report Record Number,Provider Control Type Code,Provider Number,National Provider Identifier,Report Status Code,Fiscal Year Begin Date,Fiscal Year End Date,Process Date,Initial Report Switch,Last Report Switch,The current transmittal or version number in effect for each sub-system.,Fiscal Intermediary Number,Automated Desk Review Vendor Code,Fiscal Intermediary Create Date,Utilization Code,Notice of Program Reimbursement Date,Special Indicator,Fiscal Intermediary Receipt Date
0,623132,9,010032,NaN,1,10/01/2017,11/13/2017,04/26/2018,N,N,K,10001,4,04/19/2018,F,NaN,NaN,04/16/2018
48,638926,9,010032,NaN,1,11/14/2017,06/30/2018,12/18/2018,N,N,L,10001,4,12/14/2018,F,NaN,NaN,11/29/2018
2921,651150,2,010032,NaN,1,07/01/2018,01/08/2019,07/10/2019,N,N,M,10001,4,07/02/2019,F,NaN,NaN,06/07/2019


In [13]:
hosp_df[hosp_df['Provider Number'] == '010032']

,Provider Number,FYB,FYE,STATUS,CTRL_TYPE,HOSP10_Name,Street_Addr,PO_Box,City,State,Zip_Code,County
20,010032,01-JUL-18,08-JAN-19,As Submitted,2,TANNER MEDICAL CENTER-EAST ALABAMA,1032 MAIN STREET SOUTH,NaN,WEDOWEE,AL,36278,RANDOLPH


In [14]:
hosp10_nmrc_df[hosp10_nmrc_df['Report Record Number'] == 623132] 

,Report Record Number,Worksheet Identifier,Line Number,Column Number,Item Value Number
0,623132,A000000,00100,00200,33286.0
1,623132,A000000,00100,00300,33286.0
2,623132,A000000,00100,00500,33286.0
3,623132,A000000,00100,00700,33286.0
4,623132,A000000,00400,00200,162635.0
...,...,...,...,...,...
1223,623132,S300004,01700,00100,35071.0
1224,623132,S300004,02400,00100,119947.0
1225,623132,S300004,02500,00100,42688.0
1226,623132,S300005,00100,00200,119947.0


### Join the data and format into counts

#### Filter hosp10_2018_nmrc_df records to just what we care about and index by report number.

In [15]:
filter_condition = (hosp10_nmrc_df['Worksheet Identifier'] == 'S300001') & \
                   (hosp10_nmrc_df['Column Number'].isin(value_count_list)) & \
                   (hosp10_nmrc_df['Line Number'].isin(bedtype_list))
filtered_record_df = hosp10_nmrc_df[filter_condition].set_index('Report Record Number')
filtered_record_df.count()


Worksheet Identifier    44629
Line Number             44629
Column Number           44629
Item Value Number       44629
dtype: int64

#### Join filtered numeric records, HOSP10_RPT, and HOSPITAL10 data

In [16]:
hosp_and_rpt = hosp10_rpt_df.join(hosp_df.set_index('Provider Number'), 
                                 on='Provider Number')
hosp_and_rpt_and_records = hosp_and_rpt.join(filtered_record_df, on='Report Record Number')


In [17]:
hosp_and_rpt_and_records[['Provider Number', 'Process Date', 'Report Record Number', 'Line Number', 'Column Number', 'Item Value Number']].head()

,Provider Number,Process Date,Report Record Number,Line Number,Column Number,Item Value Number
0,010032,04/26/2018,623132,00100,00200,34.0
0,010032,04/26/2018,623132,00100,00300,1496.0
0,010032,04/26/2018,623132,00100,00800,63.0
0,010032,04/26/2018,623132,01400,00200,34.0
0,010032,04/26/2018,623132,01400,00300,1496.0


#### Group the data, creating columns that contain the count values we are interested in

This step creates columns that have a 0 value for all rows in the dataframe except those that match the Line Number and Column Number for the target counts; for these rows, the value will be the count value in 'Item Number Value'.
We then perform a "groupby" operation that will sum lines up per Report Record Number and Provider Number, so that we end up with a row per report that has all the counts of interest.

In [18]:
report_cells_to_be_grouped = hosp_and_rpt_and_records.copy()

In [19]:
count_columns = []
for bed_desc, bed_key in beds_dict.items():
    for value_desc, value_key in value_count_dict.items():
        column_name = '{} {}'.format(bed_desc, value_desc)
        count_columns.append(column_name)
        def column_mapper(row):            
            if row['Line Number'] == bed_key and row['Column Number'] == value_key:
                return row['Item Value Number']
            else:
                return 0
        report_cells_to_be_grouped[column_name] = report_cells_to_be_grouped.apply(column_mapper, axis=1)

In [20]:
report_cells_to_be_grouped

,Report Record Number,Provider Control Type Code,Provider Number,National Provider Identifier,Report Status Code,Fiscal Year Begin Date,Fiscal Year End Date,Process Date,Initial Report Switch,Last Report Switch,...,Coronary Care Unit Inpatient Days,Burn ICU Staffed Beds,Burn ICU Bed Days Available,Burn ICU Inpatient Days,Surgical ICU Staffed Beds,Surgical ICU Bed Days Available,Surgical ICU Inpatient Days,Total Staffed Beds,Total Bed Days Available,Total Inpatient Days
0,623132,9,010032,NaN,1,10/01/2017,11/13/2017,04/26/2018,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,623132,9,010032,NaN,1,10/01/2017,11/13/2017,04/26/2018,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,623132,9,010032,NaN,1,10/01/2017,11/13/2017,04/26/2018,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,623132,9,010032,NaN,1,10/01/2017,11/13/2017,04/26/2018,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0
0,623132,9,010032,NaN,1,10/01/2017,11/13/2017,04/26/2018,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1496.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5703,662634,2,531310,NaN,1,07/01/2018,06/30/2019,12/31/2019,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5703,662634,2,531310,NaN,1,07/01/2018,06/30/2019,12/31/2019,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5703,662634,2,531310,NaN,1,07/01/2018,06/30/2019,12/31/2019,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
5703,662634,2,531310,NaN,1,07/01/2018,06/30/2019,12/31/2019,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9125.0,0.0


In [21]:
record_sum_columns = ['Provider Number', 'Report Record Number'] + count_columns
provider_record_sums = report_cells_to_be_grouped[record_sum_columns].groupby(
    ['Provider Number', 'Report Record Number']).sum()


### Calculate ICU totals and Occupancy rates

In [23]:
provider_record_sums['ICU Total Staffed Beds'] = provider_record_sums[icu_staffed_beds_columns].sum(axis=1)
provider_record_sums['ICU Total Bed Days Available'] = provider_record_sums[icu_bed_days_columns].sum(axis=1)
provider_record_sums['ICU Total Inpatient Days'] = provider_record_sums[icu_inpatient_days_columns].sum(axis=1)
provider_record_sums['ICU Occupancy Rate'] = provider_record_sums['ICU Total Inpatient Days']/provider_record_sums['ICU Total Bed Days Available']
provider_record_sums['Total Bed Occupancy Rate'] = provider_record_sums['Total Inpatient Days']/provider_record_sums['Total Bed Days Available']


### Join with full report data and create unique provider record by using latest report

In [58]:
full_df = hosp_and_rpt.set_index(['Provider Number', 'Report Record Number']).join(provider_record_sums)

full_df = full_df.reset_index()
full_df['Fiscal Year End Date'] = pd.to_datetime(full_df['Fiscal Year End Date'])
full_df = full_df.sort_values('Fiscal Year End Date', ascending=False)


In [59]:
# check that duplicates are dropped by Fiscal Year End Date correctly
dup_provider_nums = full_df[full_df.duplicated('Provider Number')]['Provider Number'].values
full_df.loc[full_df['Provider Number'].isin(dup_provider_nums)].sort_values('Provider Number')

,Provider Number,Report Record Number,Provider Control Type Code,National Provider Identifier,Report Status Code,Fiscal Year Begin Date,Fiscal Year End Date,Process Date,Initial Report Switch,Last Report Switch,...,Surgical ICU Bed Days Available,Surgical ICU Inpatient Days,Total Staffed Beds,Total Bed Days Available,Total Inpatient Days,ICU Total Staffed Beds,ICU Total Bed Days Available,ICU Total Inpatient Days,ICU Occupancy Rate,Total Bed Occupancy Rate
34,010022,637570,4,NaN,1,04/01/2018,2018-05-31,11/28/2018,N,N,...,0.0,0.0,45.0,1395.0,411.0,0.0,0.0,0.0,NaN,0.294624
4639,010022,660348,4,NaN,1,06/01/2018,2019-06-30,12/11/2019,N,N,...,0.0,0.0,45.0,17775.0,4311.0,0.0,0.0,0.0,NaN,0.242532
0,010032,623132,9,NaN,1,10/01/2017,2017-11-13,04/26/2018,N,N,...,0.0,0.0,34.0,1496.0,63.0,0.0,0.0,0.0,NaN,0.042112
2921,010032,651150,2,NaN,1,07/01/2018,2019-01-08,07/10/2019,N,N,...,0.0,0.0,15.0,2880.0,653.0,0.0,0.0,0.0,NaN,0.226736
48,010032,638926,9,NaN,1,11/14/2017,2018-06-30,12/18/2018,N,N,...,0.0,0.0,15.0,3435.0,1025.0,0.0,0.0,0.0,NaN,0.298399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3844,521315,656021,2,NaN,5,10/01/2017,2017-12-31,11/06/2019,N,N,...,0.0,0.0,25.0,2300.0,1233.0,0.0,0.0,0.0,NaN,0.536087
4637,530025,660342,11,NaN,1,07/01/2018,2019-06-30,12/10/2019,N,N,...,0.0,0.0,66.0,24090.0,5329.0,4.0,1460.0,329.0,0.225342,0.221212
80,530025,640598,11,NaN,1,01/01/2018,2018-06-30,01/24/2019,N,N,...,0.0,0.0,66.0,11946.0,2416.0,4.0,724.0,169.0,0.233425,0.202243
3599,673058,654708,4,NaN,1,06/26/2018,2019-04-30,10/16/2019,N,N,...,0.0,0.0,36.0,13140.0,5588.0,0.0,0.0,0.0,NaN,0.425266


In [60]:
full_df = full_df.drop_duplicates('Provider Number')

In [61]:
# check that duplicates are dropped by Fiscal Year End Date correctly
full_df.loc[full_df['Provider Number'].isin(dup_provider_nums)].sort_values('Provider Number')

,Provider Number,Report Record Number,Provider Control Type Code,National Provider Identifier,Report Status Code,Fiscal Year Begin Date,Fiscal Year End Date,Process Date,Initial Report Switch,Last Report Switch,...,Surgical ICU Bed Days Available,Surgical ICU Inpatient Days,Total Staffed Beds,Total Bed Days Available,Total Inpatient Days,ICU Total Staffed Beds,ICU Total Bed Days Available,ICU Total Inpatient Days,ICU Occupancy Rate,Total Bed Occupancy Rate
4639,010022,660348,4,NaN,1,06/01/2018,2019-06-30,12/11/2019,N,N,...,0.0,0.0,45.0,17775.0,4311.0,0.0,0.0,0.0,NaN,0.242532
2921,010032,651150,2,NaN,1,07/01/2018,2019-01-08,07/10/2019,N,N,...,0.0,0.0,15.0,2880.0,653.0,0.0,0.0,0.0,NaN,0.226736
2922,010069,651152,9,NaN,1,12/01/2017,2018-12-31,07/10/2019,N,N,...,0.0,0.0,30.0,11880.0,5116.0,5.0,1980.0,842.0,0.425253,0.430640
3479,010126,654064,8,NaN,1,04/01/2018,2019-03-31,09/20/2019,N,N,...,0.0,0.0,97.0,35405.0,6513.0,9.0,3285.0,1006.0,0.306240,0.183957
2611,030137,650635,4,NaN,1,07/25/2018,2018-12-31,07/10/2019,N,N,...,0.0,0.0,49.0,7840.0,2492.0,6.0,960.0,365.0,0.380208,0.317857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,521302,649520,2,NaN,1,01/01/2018,2018-12-31,06/19/2019,N,N,...,0.0,0.0,18.0,6570.0,4047.0,0.0,0.0,0.0,NaN,0.615982
1973,521314,649523,2,NaN,1,01/01/2018,2018-12-31,06/19/2019,N,N,...,0.0,0.0,25.0,9125.0,4840.0,0.0,0.0,0.0,NaN,0.530411
1974,521315,649524,2,NaN,1,01/01/2018,2018-12-31,06/19/2019,N,N,...,0.0,0.0,25.0,9125.0,4705.0,0.0,0.0,0.0,NaN,0.515616
4637,530025,660342,11,NaN,1,07/01/2018,2019-06-30,12/10/2019,N,N,...,0.0,0.0,66.0,24090.0,5329.0,4.0,1460.0,329.0,0.225342,0.221212


### Generate final dataframe of provider information

In [62]:
full_df.to_csv('full_df.csv', header=True)

In [63]:
final_columns = list(hosp_df.columns.values) + list(provider_record_sums.columns.values)

In [64]:
final_df = full_df[final_columns]

In [65]:
final_df.head()

,Provider Number,FYB,FYE,STATUS,CTRL_TYPE,HOSP10_Name,Street_Addr,PO_Box,City,State,...,Surgical ICU Bed Days Available,Surgical ICU Inpatient Days,Total Staffed Beds,Total Bed Days Available,Total Inpatient Days,ICU Total Staffed Beds,ICU Total Bed Days Available,ICU Total Inpatient Days,ICU Occupancy Rate,Total Bed Occupancy Rate
5502,102010,01-SEP-18,31-AUG-19,As Submitted,4,KINDRED HOSPITAL SOUTH FLORIDA,1516 EAST LAS OLAS BOULEVARD,NaN,FT. LAUDERDALE,FL,...,0.0,0.0,214.0,78110.0,49247.0,23.0,8395.0,5560.0,0.662299,0.630483
5453,494022,01-SEP-18,31-AUG-19,As Submitted,4,POPLAR SPRINGS HOSPITAL,350 POPLAR DRIVE,NaN,PETERSBURG,VA,...,0.0,0.0,124.0,45260.0,28428.0,0.0,0.0,0.0,NaN,0.628104
5505,142009,01-SEP-18,31-AUG-19,As Submitted,4,KINDRED HOSPITAL CHICAGO CENTRAL,4058 WEST MELROSE STREET,NaN,CHICAGO,IL,...,0.0,0.0,187.0,68255.0,22812.0,0.0,0.0,0.0,NaN,0.334217
5504,142006,01-SEP-18,31-AUG-19,As Submitted,4,KINDRED HOSPITAL SYCAMORE,225 EDWARD STREET,NaN,SYCAMORE,IL,...,0.0,0.0,69.0,25185.0,9925.0,0.0,0.0,0.0,NaN,0.394084
5500,052038,01-SEP-18,31-AUG-19,As Submitted,4,KINDRED HOSPITAL LA MIRADA,14900 E. IMPERIAL HIGHWAY,NaN,LA MIRADA,CA,...,0.0,0.0,216.0,78840.0,48227.0,19.0,6935.0,4951.0,0.713915,0.611707


### Drop bad facility data

In [66]:
# Drop PARKVIEW MEDICAL CENTER which reports over 2 million staffed beds.
final_df = final_df[final_df['Provider Number'] != '060020']

### Match data to geocoded facility information

These points are geocoded from the origina v1 notebook `usa_hcris2018_facilitybedcounts_20200313_v1.ipynb`.

Notes from original GeoCoding:

- geocoding first with google maps which gave best results, fall back to mapbox geocode and then search_str without street address
- TODO: figure out why geocoder with gmaps stopped sending requests halfway through, temp switch to directly using gmaps api

In [67]:
geojson = json.loads(open('/opt/src/data/usa_facilities_hcris_geocoded.geojson').read())

In [68]:
## Key our data by Provider Number
final_feature_data = final_df.to_dict(orient='records')
final_feature_data_keyed = dict((r['Provider Number'], r) for r in final_feature_data)

## Use the HOSPITAL10 data to account for facilities that don't have report data
hosp_feature_data = hosp_df.to_dict(orient='records')
hosp_feature_data_keyed = dict((r['Provider Number'], r) for r in hosp_feature_data)

#### Create empty facility information for facilities without HCRIS records

In [69]:
nodata_properties = copy(final_feature_data[0])
for key in nodata_properties:
    if type(nodata_properties[key]) is int:
        nodata_properties[key] = 0
    elif type(nodata_properties[key]) is float:
        nodata_properties[key] = 0.0
    else:
        nodata_properties[key] = 'NoData'

In [70]:
def generate_properties(provider_number):
    if provider_number in final_feature_data_keyed:
        new_properties = final_feature_data_keyed[provider_number]
        return new_properties, True
    else:
        new_properties = copy(nodata_properties)
        new_properties['Provider Number'] = provider_number
        if provider_number in hosp_feature_data_keyed:
            hosp_info = hosp_feature_data_keyed[provider_number]
            for key in new_properties:
                if key in hosp_info:
                    new_properties[key] = hosp_info[key]
        return new_properties, False

#### Replace properties from geocoded facilities with calculated HRCIS properties

In [71]:
facilities_with_data = 0
facilities_without_data = 0
for feature in geojson['features']:
    provider_number = feature['properties']['Provider Number']
    new_props, found = generate_properties(provider_number)
    if found:
        facilities_with_data += 1
    else:
        facilities_without_data += 1
        
    feature['properties'] = new_props
    
print('{} Found, {} not found'.format(facilities_with_data, facilities_without_data))

5603 Found, 1058 not found


In [72]:
geojson['features'][0]

{'type': 'Feature',
 'properties': {'Provider Number': '010001',
  'FYB': '01-OCT-17',
  'FYE': '30-SEP-18',
  'STATUS': 'As Submitted',
  'CTRL_TYPE': 9,
  'HOSP10_Name': 'SOUTHEAST HEALTH MEDICAL CENTER',
  'Street_Addr': '1108 ROSS CLARK CIRCLE',
  'PO_Box': '6987',
  'City': 'DOTHAN',
  'State': 'AL',
  'Zip_Code': '36301',
  'County': 'HOUSTON',
  'Hospital Adult and Peds Staffed Beds': 271.0,
  'Hospital Adult and Peds Bed Days Available': 98915.0,
  'Hospital Adult and Peds Inpatient Days': 78031.0,
  'Intensive Care Unit Staffed Beds': 40.0,
  'Intensive Care Unit Bed Days Available': 14600.0,
  'Intensive Care Unit Inpatient Days': 11992.0,
  'Coronary Care Unit Staffed Beds': 0.0,
  'Coronary Care Unit Bed Days Available': 0.0,
  'Coronary Care Unit Inpatient Days': 0.0,
  'Burn ICU Staffed Beds': 0.0,
  'Burn ICU Bed Days Available': 0.0,
  'Burn ICU Inpatient Days': 0.0,
  'Surgical ICU Staffed Beds': 0.0,
  'Surgical ICU Bed Days Available': 0.0,
  'Surgical ICU Inpatient 

### Write usa_hospital_beds_hcris2018 GeoJSON

In [73]:
with open('/opt/src/data/usa_hospital_beds_hcris2018_v2.geojson', 'w') as f:
    f.write(json.dumps(geojson, indent=4))